In [1]:
import json
from pathlib import Path

import pandas as pd

In [2]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/musique-sweep-manual-new/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[1]

33 experiments


{'commit': '05eab74f7847aec6ae628588ebee83a0bb114fad',
 'id': '55ba0a2e8beb31a6806682a88bf7b5820c568cf7',
 'name': 'losel-keep',
 'params': {'dataset': {'path': 'bdsaglam/musique-sweep',
   'name': 'answerable',
   'split': 'train'},
  'qa': {'model': 'llama-3-70b-tgi',
   'temperature': 0.5,
   'system_prompt': 'cte/format-triples-ere-few.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'manual/cte-triples.json',
   'n_shot': 0,
   'n_sc': 1},
  'run': 1},
 'metrics': {}}

In [3]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
mask = (
    (df["params.dataset.path"] == "bdsaglam/musique-sweep")
    & (df["metrics.f1"] > 0.1)
)

df = df.loc[mask].copy()
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=metric_cols[:2], inplace=True, how="any")
df.drop_duplicates(subset=param_cols + metric_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

11 experiments after preprocessing


In [4]:
def parse_qa_technique(row):
    base_technique = row["params.qa.system_prompt"].split("/")[0].upper()
    sc = int(row["params.qa.n_sc"])
    sc_suffix = f"-SC({sc})" if sc > 1 else ""
    return f"{base_technique}{sc_suffix}"

df["params.qa.technique"] = df.apply(parse_qa_technique, axis=1)

In [5]:
def sorted_tuple(x):
    return tuple(sorted(x))


run_counts = (
    df.groupby(
        [
            "params.qa.system_prompt",
            "params.qa.user_prompt_template",
            "params.qa.few_shot_examples",
            "params.qa.n_shot",
            "params.qa.n_sc",
            "params.qa.temperature",
        ]
    )["params.run"]
    .aggregate(sorted_tuple)
    .reset_index()
)
run_counts.loc[run_counts["params.run"].map(len) != 3]

,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.n_shot,params.qa.n_sc,params.qa.temperature,params.run
0,cok/adapted.txt,cq.txt,manual/cok.json,0,1,0.0,"(2,)"
1,cok/adapted.txt,cq.txt,manual/cok.json,0,1,0.5,"(1,)"
2,cok/adapted.txt,cq.txt,manual/cok.json,1,1,0.0,"(1,)"
3,cte/format-sro-triples-few.txt,cq.txt,manual/cte-triples.json,0,1,0.0,"(2,)"
4,cte/format-sro-triples-few.txt,cq.txt,manual/cte-triples.json,0,1,0.5,"(2,)"
5,cte/format-sro-triples-few.txt,cq.txt,manual/cte-triples.json,1,1,0.0,"(2,)"
6,cte/format-sro-triples-few.txt,cq.txt,manual/cte-triples.json,1,1,0.5,"(1,)"
7,cte/format-triples-ere-few.txt,cq.txt,manual/cte-triples.json,0,1,0.0,"(2,)"
8,cte/format-triples-ere-few.txt,cq.txt,manual/cte-triples.json,1,1,0.0,"(1,)"
9,cte/format-triples-ere-few.txt,cq.txt,manual/cte-triples.json,1,1,0.5,"(1,)"


In [6]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.qa.system_prompt
	cok/adapted.txt
	cte/format-sro-triples-few.txt
	cte/format-triples-ere-few.txt
	cte/format-triples-few.txt
- params.qa.user_prompt_template
	cq.txt
- params.qa.few_shot_examples
	manual/cok.json
	manual/cte-triples.json
- params.qa.n_shot
	0
	1
- params.qa.n_sc
	1
- params.qa.temperature
	0.0
	0.5
- params.run
	(2,)
	(1,)


In [7]:
df.to_json('exps.jsonl', orient='records', lines=True)